# Inverse kinematics

This example shows how to solve the inverse kinematics problem for the right foot treated as end effector. The example employs s simple Jacobian-based iterative algorithm, which is called closed-loop kinematics (CLIK).

The script is needed to compute parameters for SRDF.

REF: https://gepettoweb.laas.fr/doc/stack-of-tasks/pinocchio/master/doxygen-html/md_doc_b-examples_d-inverse-kinematics.html

# Libraries

As the first step we need to import the necessary libraries and create the Pinocchio Model and Data objects. Here we use URDF to build the model.

In [1]:
import os
import numpy as np
from numpy.linalg import norm, solve
import pinocchio
from pinocchio.visualize import MeshcatVisualizer

URDF = os.path.join(os.path.abspath(''), "h1_description/h1.urdf")
MESH = os.path.join(os.path.abspath(''), "")
robot = pinocchio.RobotWrapper.BuildFromURDF(URDF,MESH)
model = robot.model
data = model.createData()

# Visualization

Within this example we use meshcat to visualize the model and its state. 

In [2]:
viz = MeshcatVisualizer(model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()
q0 = pinocchio.neutral(model)
viz.display(q0)
viz.displayVisuals(True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
find_ffmpeg failed, using the integrated library.
Opening in existing browser session.


The end effector corresponds to 'left_ankle_link'

In [3]:
EE_NAME = 'left_ankle_link'
EE_ID = model.getFrameId(EE_NAME)

and its desired pose is given as

In [4]:
oMdes = pinocchio.SE3(np.eye(3), np.array([.039468, .20286, -.75]))

Next, we define an initial configuration. This is the starting point of the algorithm. A priori, any valid configuration would work. We decided to use the robot's neutral configuration, returned by algorithm neutral. For an open chain as the one in this case, it simply corresponds to an all-zero vector, but using this method generalizes well to more complex kinds of robots, ensuring validity.

In [5]:
q = pinocchio.neutral(model)

Next, we set some computation-related values corresponding to the desired position precision (we will see later what it exactly means), a maximum number of iterations (to avoid infinite looping in case the position is not reachable), a positive "time step" defining the convergence rate, and a fixed damping factor for the pseudoinversion (see below).

In [6]:
eps = 1e-2
IT_MAX = 1000
DT = 1e-1
damp = 1e-3

Then, we begin the iterative process.

In [7]:
i = 0
while True:
    # At each iteration, we begin by computing the forward kinematics
    pinocchio.framesForwardKinematics(model, data, q)
    # Next, we compute the error between the desired pose d and 
    # the current one i, expressed in the current joint frame i:
    iMd = data.oMf[EE_ID].actInv(oMdes)
    err = pinocchio.log(iMd).vector # in joint frame
    if norm(err) < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pinocchio.computeFrameJacobian(model, data, q, EE_ID)
    J = -np.dot(pinocchio.Jlog6(iMd.inverse()), J)
    v = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pinocchio.integrate(model, q, v * DT)
    if not i % 10:
        print("%d: error = %s" % (i, err.T))
    viz.display(q)
    viz.sleep(0.05)
    i += 1

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)

0: error = [0.00000000e+00 2.77555756e-17 2.24200000e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
10: error = [ 3.47973973e-06 -7.80128980e-03  2.22885722e-01 -1.13595655e-02
 -2.08480438e-06  3.28197637e-04]
20: error = [ 3.97928468e-06 -4.47106953e-03  1.05652731e-01 -7.12018001e-03
 -3.43932535e-04  1.95672178e-04]
30: error = [ 1.15229639e-06 -1.55551785e-03  3.75233071e-02 -2.62012361e-03
 -2.06120819e-04  6.81231548e-05]
40: error = [ 2.46065250e-07 -5.50408348e-04  1.33568075e-02 -9.46488465e-04
 -9.93274508e-05  2.40676524e-05]
Convergence achieved!

result: [-1.767081754997438e-05, 0.0006964058793829477, -0.7493938077395171, 1.4987963822508579, -0.7493242191876197, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

final error: [ 1.41272067e-07 -4.03733047e-04  9.79882407e-03 -6.96406571e-04
 -7.83491675e-05  1.76435333e-05]
